In [20]:
import os
import pandas as pd
import numpy as np

In [21]:
drivers_df = pd.read_csv('F1KaggleData/drivers.csv')
races_df = pd.read_csv('F1KaggleData/races.csv')
circuits_df = pd.read_csv('F1KaggleData/circuits.csv')
results_df = pd.read_csv('F1KaggleData/results.csv')


In [22]:
display(drivers_df.sample(5))
# display(races_df.head(5))
# display(circuits_df.head(5))
# display(results_df.head(5))


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
185,186,acheson,NaN,NaN,Kenny,Acheson,27/11/1957,British,http://en.wikipedia.org/wiki/Kenny_Acheson
320,321,bell,NaN,NaN,Derek,Bell,31/10/1941,British,http://en.wikipedia.org/wiki/Derek_Bell_(auto_...
128,129,grouillard,NaN,NaN,Olivier,Grouillard,02/09/1958,French,http://en.wikipedia.org/wiki/Olivier_Grouillard
105,106,beretta,NaN,NaN,Olivier,Beretta,23/11/1969,Monegasque,http://en.wikipedia.org/wiki/Olivier_Beretta
472,472,harris,NaN,NaN,Mike,Harris,25/05/1939,South African,http://en.wikipedia.org/wiki/Mike_Harris_(race...


In [23]:
drivers_df.loc[drivers_df['driverRef'] == 'hamilton']

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


In [24]:
def driver_finishing_in_top_n(driverId: int, n: int, results_df:pd.DataFrame):

    positions = [float(i) for i in list(range(1, n+1))]
    # display(positions)
    positions_df = results_df[results_df['position'].isin(positions)]
    return positions_df.loc[positions_df['driverId'] == driverId]
    
    
    
driver_results_df = driver_finishing_in_top_n(1, 1, results_df)
# display(driver_results_df.sample(10))

wins_per_constructor_df = driver_results_df['constructorId'].value_counts().to_frame()
wins_per_constructor_df = wins_per_constructor_df.reset_index()
wins_per_constructor_df.rename(columns = {'index':'constructorId', 'constructorId':'frequency'}, inplace = True) 
display(wins_per_constructor_df)


,constructorId,frequency
0,131,41
1,1,21


In [25]:
constructor_df = pd.read_csv('F1KaggleData/constructors.csv')
display(constructor_df.sample(5))

constructor_df_lite = constructor_df[['constructorId', 'constructorRef']]
display(constructor_df_lite.sample(10))

display()

,constructorId,constructorRef,name,nationality,url,Unnamed: 5
91,93,re,RE,Rhodesian,http://en.wikipedia.org/wiki/RE_%28automobile%29,NaN
55,57,ensign,Ensign,British,http://en.wikipedia.org/wiki/Ensign_%28racing_...,NaN
149,151,era,ERA,British,http://en.wikipedia.org/wiki/English_Racing_Au...,NaN
83,85,bellasi,Bellasi,Swiss,http://en.wikipedia.org/wiki/Bellasi,NaN
65,67,penske,Penske,American,http://en.wikipedia.org/wiki/Penske_Racing,NaN


,constructorId,constructorRef
194,197,brm-ford
16,17,jordan
149,151,era
178,181,brabham-brm
81,83,politoys
18,19,jaguar
113,115,vhristensen
38,39,ags
137,139,bromme
79,81,tecno


In [26]:
def get_constructor_name(constructorId: int) -> str:
    
    return constructor_df.loc[constructor_df['constructorId']==constructorId]



In [27]:
def get_driver_ref(driverId: int, drivers_df: pd.DataFrame) -> str:
    
    return drivers_df.loc[drivers_df['driverId']==driverId]['driverRef'].values[0]

def get_driver_id(driver_ref: str, drivers_df: pd.DataFrame) -> int:
    
    return drivers_df.loc[drivers_df['driverRef']==driver_ref]['driverId'].values[0]

def driver_finishing_in_top_n(driverId: int, n: int, 
                              results_df:pd.DataFrame, 
                              constructor_lite_df:pd.DataFrame):

    positions = [float(i) for i in list(range(1, n+1))]
    # display(positions)
    positions_df = results_df[results_df['position'].isin(positions)]
    driver_results_df = positions_df.loc[positions_df['driverId'] == driverId]

    wins_per_constructor_df = driver_results_df['constructorId'].value_counts().to_frame()
    wins_per_constructor_df = wins_per_constructor_df.reset_index()
    wins_per_constructor_df.rename(columns = {'index':'constructorId', 'constructorId':'frequency'}, inplace = True) 
    wins_per_constructor_df = pd.merge(wins_per_constructor_df, constructor_df_lite, on='constructorId')
    wins_per_constructor_df.drop('constructorId', axis=1, inplace=True)
    
    wins_per_constructor_df.set_index([[get_driver_ref(driverId, drivers_df)]*len(wins_per_constructor_df)], inplace=True)
    
    return wins_per_constructor_df

    
display(driver_finishing_in_top_n(get_driver_id('hamilton', drivers_df), 3, results_df, constructor_df_lite))
hamilton_podiums = driver_finishing_in_top_n(get_driver_id('hamilton', drivers_df), 3, results_df, constructor_df_lite)
display(driver_finishing_in_top_n(get_driver_id('vettel', drivers_df), 3, results_df, constructor_df_lite))
display(driver_finishing_in_top_n(get_driver_id('alonso', drivers_df), 3, results_df, constructor_df_lite))



,frequency,constructorRef
hamilton,68,mercedes
hamilton,49,mclaren


,frequency,constructorRef
vettel,65,red_bull
vettel,33,ferrari
vettel,1,toro_rosso


,frequency,constructorRef
alonso,44,ferrari
alonso,41,renault
alonso,12,mclaren


In [28]:
# for drivers active in a particular season, lets see all of their wins and podiums? 

